In [ ]:
import numpy as np 
import pandas as pd 
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import keras
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, InceptionResNetV2, ResNet50, Xception
import cv2
from PIL import Image

In [ ]:
path = '../input/plant-pathology-2021-fgvc8/'
train_dir = path + 'train_images/'
test_dir = path + 'test_images/'

In [ ]:
df = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')

In [ ]:
df.head()

In [ ]:
df.labels.value_counts()

In [ ]:
df['labels'].unique()

In [ ]:
df['labels'] = df['labels'].astype(str)

In [ ]:
plt.figure(figsize=(8,5))
sns.countplot(data = df,y='labels')

In [ ]:
def plot_examples(label):
    fig, ax = plt.subplots(1, 5, figsize=(25, 15))
    ax = ax.ravel()
    for i in range(5):
        idx = df[df['labels']==label].index[i]
        image = cv2.imread(train_dir+df.loc[idx, 'image'])
        
        image =cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        ax[i].imshow(image)
        ax[i].set_title(label)
        ax[i].set_xticklabels([])
        ax[i].set_yticklabels([])

In [ ]:
for labels in list(df['labels'].unique()):
    plot_examples(labels)

In [ ]:
train_datagen  = ImageDataGenerator(rotation_range = 180,
                                   width_shift_range = 0.1,
                                   height_shift_range = 0.1,
                                   horizontal_flip = True,
                                   rescale = 1./255,
                                   zoom_range = 0.2,
                                   validation_split = 0.2)
test_datagen  = ImageDataGenerator(rescale = 1./255,
                                   validation_split = 0.2)

In [ ]:
train_generator = train_datagen.flow_from_dataframe(dataframe = df,
                                                   directory = train_dir,
                                                   target_size = (150,150),
                                                   x_col = 'image',
                                                   y_col = 'labels',
                                                   batch_size = 256,
                                                   color_mode = 'rgb',
                                                   class_mode = 'categorical',
                                                   subset = 'training')

test_generator = test_datagen.flow_from_dataframe(dataframe = df,
                                                 directory = train_dir,
                                                 target_size = (150,150),
                                                 x_col = 'image',
                                                 y_col = 'labels',
                                                 batch_size = 256,
                                                 color_mode = 'rgb',
                                                 class_mode = 'categorical',
                                                 subset = 'validation')

In [ ]:
"""model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation = 'relu', padding = 'Same',input_shape = [150,150,3]),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.25),
    
    tf.keras.layers.Conv2D(128, (3,3), activation = 'relu',padding = 'Same'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25),
    
    tf.keras.layers.Conv2D(128, (3,3), activation = 'relu',padding = 'Same'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25),
    
    tf.keras.layers.Conv2D(128, (3,3), activation = 'relu',padding = 'Same'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25),
    
    tf.keras.layers.Conv2D(256, (3,3), activation = 'relu',padding = 'Same'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25),
    
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024,activation = 'relu'),
    tf.keras.layers.Dense(12, activation = 'softmax')
])"""

In [ ]:
model = tf.keras.Sequential()
model.add(VGG16(include_top = False,weights = '../input/keras-pretrained-models/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5',input_shape= (150,150,3)))
model.add(MaxPool2D(padding = 'same'))
model.add(BatchNormalization(axis=-1))
model.add(Flatten())
model.add(Dense(12,activation = 'softmax'))

In [ ]:
from tensorflow.keras.optimizers import RMSprop,Adam
epochs = 10
batch_size = 256
optimizer = Adam(lr = 0.001)
model.compile(loss = 'categorical_crossentropy',
              optimizer = optimizer,
             metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit_generator(train_generator,epochs = epochs,validation_data = test_generator)